In [26]:
# ==== 1) Import ====
import re
import pandas as pd
import numpy as np

pd.set_option("display.max_colwidth", 120)

METRICS = ["context_recall", "answer_correctness", "faithfulness", "answer_relevancy"]

def _normalize_user_input(x: str) -> str:
    """Chuẩn hoá để khớp user_input giữa 2 file (giảm sai khác do khoảng trắng/newline)."""
    if pd.isna(x):
        return ""
    x = str(x)
    x = x.replace("\u00a0", " ")  # non-breaking space
    x = re.sub(r"\s+", " ", x).strip()
    return x

In [27]:
proposed_path = 'chatbot/evaluation_proposed_final.csv'
baseline_path = 'baseline_rag/ragas_evaluation.csv'

df_proposed = pd.read_csv(proposed_path)
df_baseline = pd.read_csv(baseline_path)

print("df_proposed shape:", df_proposed.shape)
print("df_baseline shape:", df_baseline.shape)

display(df_proposed.head(3))
display(df_baseline.head(3))


df_proposed shape: (64, 8)
df_baseline shape: (49, 8)


,user_input,retrieved_contexts,response,reference,context_recall,answer_correctness,faithfulness,answer_relevancy
0,Actisô thuộc họ thực vật nào?,['Tên cây: Actisô\nTên gọi khác: Artichant\nTên khoa học: Cynara scolymus L.\nHọ: Asteraceae\nĐặc điểm thực vật: Cây...,Actisô thuộc họ Asteraceae.\n\nNguồn: id=data\raw\cay-rau-lam-thuoc\cay-rau-lam-thuoc.md:#0,Họ Cúc (Asteraceae/Compositae).,1.0,0.677033,1.0,0.855611
1,"Trong cụm hoa Actisô, glucid chủ yếu là chất gì?",['Tên cây: Actisô\nTên gọi khác: Artichant\nTên khoa học: Cynara scolymus L.\nHọ: Asteraceae\nĐặc điểm thực vật: Cây...,"Trong cụm hoa Actisô, glucid chủ yếu là inulase.\n\nNguồn: id=data\raw\cay-rau-lam-thuoc\cay-rau-lam-thuoc.md:#0",Glucid chủ yếu là inulase.,0.0,0.931768,0.0,0.697834
2,Tên khoa học của cây Bầu là gì?,"['Tên cây: Bầu đất\nTên gọi khác: Kim thất, Rau lúi, Rau bầu đất\nTên khoa học: Gynura procumbens (Lour.) Merr. = Gy...",Tên khoa học của cây Bầu là Lagenaria siceraria (Molina) Standl.\n\nNguồn: #2 id=data\raw\cay-rau-lam-thuoc\cay-rau-...,Lagenaria siceraria (Molina) Standl.,1.0,0.875777,1.0,0.615235


,user_input,retrieved_contexts,response,reference,context_recall,answer_correctness,faithfulness,answer_relevancy
0,Bromelin trong Dứa có tác dụng chính gì về tiêu hoá?,"['#### (33) DứA\nDứa, Thơm, Khóm – Ananas comosus (L.) Merr., thuộc họ Dứa – Bromeliaceae. Dứa là loại cây thảo có t...",Bromelin trong dứa có tác dụng chính về tiêu hóa là giúp tiêu hóa các protein. Chất này hoạt động giống như pepsin t...,Giúp tiêu hoá protein (tương tự pepsin).,1.0,0.667029,0.666667,0.712332
1,"Ngoài tiêu hoá, Bromelin còn có tác dụng gì được nêu?","['## Page 349\n\n2. 2.1.Tim mạch\n\nNhịp tim chậm, hạ huyết áp, suy hô hấp đã được báo cáo khi quá liều ở trẻ em,\nn...","Dựa trên thông tin được cung cấp, Bromelin không có tác dụng nào được đề cập ngoài tiêu hóa. Để trả lời chính xác câ...","Tiêu viêm, giảm phù và hỗ trợ điều trị vết thương.",0.0,0.126045,0.000000,0.000000
2,Bài thuốc chữa sỏi thận từ Rau ngổ dùng liều tươi khoảng bao nhiêu?,"['## 125 RAU NGỔ\nRau ngổ hay Rau om – Limnophila aromatica (Lour.) Merr., thuộc họ Hoa mõm sói – Scrophulariaceae. ...","Theo thông tin được cung cấp, bài thuốc chữa sỏi thận từ Rau ngổ dùng liều tươi khoảng 20-30g cây tươi giã ra, thêm ...",20–30g Rau ngổ tươi giã thêm nước uống (hoặc dùng khô sắc uống).,1.0,0.791345,1.000000,0.751255


In [28]:
required_cols = set(["user_input"] + METRICS)

missing_p = required_cols - set(df_proposed.columns)
missing_b = required_cols - set(df_baseline.columns)
assert not missing_p, f"eval_proposed thiếu cột: {sorted(missing_p)}"
assert not missing_b, f"eval_baseline thiếu cột: {sorted(missing_b)}"

for m in METRICS:
    df_proposed[m] = pd.to_numeric(df_proposed[m], errors="coerce")
    df_baseline[m] = pd.to_numeric(df_baseline[m], errors="coerce")

# key để khớp câu hỏi
df_proposed["_key"] = df_proposed["user_input"].map(_normalize_user_input)
df_baseline["_key"] = df_baseline["user_input"].map(_normalize_user_input)

# loại bỏ key rỗng
df_proposed = df_proposed[df_proposed["_key"] != ""].copy()
df_baseline = df_baseline[df_baseline["_key"] != ""].copy()

print("After cleaning empty user_input:")
print("df_proposed shape:", df_proposed.shape)
print("df_baseline shape:", df_baseline.shape)


After cleaning empty user_input:
df_proposed shape: (64, 9)
df_baseline shape: (49, 9)


In [29]:
# ==== 6) Tính trung bình (3 dòng) ====
# (A) Trung bình toàn bộ proposed
proposed_mean = df_proposed[METRICS].mean(numeric_only=True)

# (B) Trung bình toàn bộ baseline
baseline_mean = df_baseline[METRICS].mean(numeric_only=True)

# (C) Trung bình trên các câu chung (inner join theo _key)
# Ta lấy mean cho các metrics và 'first' cho cột response để so sánh text
p_g = df_proposed.groupby("_key").agg({
    **{m: "mean" for m in METRICS},
    "response": "first"
}).reset_index()

b_g = df_baseline.groupby("_key").agg({
    **{m: "mean" for m in METRICS},
    "response": "first"
}).reset_index()

common = p_g.merge(b_g, on="_key", suffixes=("_proposed", "_baseline"), how="inner")
n_common = common.shape[0]

if n_common == 0:
    common_avg = pd.Series({m: np.nan for m in METRICS})
else:
    per_question_avg = pd.DataFrame({
        m: (common[f"{m}_proposed"] + common[f"{m}_baseline"]) / 2.0
        for m in METRICS
    })
    common_avg = per_question_avg.mean()

summary = pd.DataFrame(
    [proposed_mean, baseline_mean, common_avg],
    index=["mean_eval_proposed", "mean_eval_baseline", "mean_common_questions_(avg_of_two)"],
)

# Thêm cột đếm để dễ kiểm tra
summary.insert(0, "n_rows", [len(df_proposed), len(df_baseline), n_common])

summary


,n_rows,context_recall,answer_correctness,faithfulness,answer_relevancy
mean_eval_proposed,64,0.395833,0.413441,0.720064,0.383967
mean_eval_baseline,49,0.653061,0.564395,0.752743,0.638751
mean_common_questions_(avg_of_two),31,0.548387,0.524476,0.723373,0.552092


In [30]:
# ==== 6.1) Phân tích chi tiết: So sánh proposed vs baseline trên cùng tập câu hỏi ====
if n_common > 0:
    diff_df = common[["_key", "response_proposed", "response_baseline"]].copy()
    diff_df.columns = ["user_input", "response_proposed", "response_baseline"]
    
    for m in METRICS:
        # Tỷ lệ cải thiện
        diff_df[f"{m}_diff"] = common[f"{m}_proposed"] - common[f"{m}_baseline"]
        
        # Thống kê số lượng
        better = (diff_df[f"{m}_diff"] > 0.001).sum()  # dùng ngưỡng nhỏ để tránh sai số float
        worse = (diff_df[f"{m}_diff"] < -0.001).sum()
        equal = n_common - better - worse
        
        print(f"Metric: {m}")
        print(f"  - Cải thiện (Proposed > Baseline): {better}")
        print(f"  - Giảm sút (Proposed < Baseline): {worse}")
        print(f"  - Không đổi: {equal}")
        print("-" * 30)

    # Tổng hợp sự thay đổi trung bình trên tất cả metrics
    diff_cols = [f"{m}_diff" for m in METRICS]
    diff_df["avg_diff"] = diff_df[diff_cols].mean(axis=1)
    
    print("\n>>> TOP 5 CẢI THIỆN NHIỀU NHẤT (dựa trên trung bình các metrics):")
    display(diff_df.sort_values("avg_diff", ascending=False).head(5)[["user_input", "avg_diff"] + diff_cols])
    
    print("\n>>> TOP 5 GIẢM SÚT NHIỀU NHẤT (dựa trên trung bình các metrics):")
    display(diff_df.sort_values("avg_diff", ascending=True).head(5)[["user_input", "avg_diff"] + diff_cols])
else:
    print("Không có câu hỏi chung để so sánh chi tiết.")


Metric: context_recall
  - Cải thiện (Proposed > Baseline): 2
  - Giảm sút (Proposed < Baseline): 10
  - Không đổi: 19
------------------------------
Metric: answer_correctness
  - Cải thiện (Proposed > Baseline): 13
  - Giảm sút (Proposed < Baseline): 17
  - Không đổi: 1
------------------------------
Metric: faithfulness
  - Cải thiện (Proposed > Baseline): 6
  - Giảm sút (Proposed < Baseline): 10
  - Không đổi: 15
------------------------------
Metric: answer_relevancy
  - Cải thiện (Proposed > Baseline): 7
  - Giảm sút (Proposed < Baseline): 17
  - Không đổi: 7
------------------------------

>>> TOP 5 CẢI THIỆN NHIỀU NHẤT (dựa trên trung bình các metrics):


,user_input,avg_diff,context_recall_diff,answer_correctness_diff,faithfulness_diff,answer_relevancy_diff
29,"Trong ẩm thực, bộ phận nào của Bầu đất thường dùng nấu canh?",0.369108,1.0,0.572495,0.000000,-0.096064
27,"Theo y học cổ truyền, Hải tảo có tác dụng gì đối với đờm?",0.277005,0.0,0.585155,0.500000,0.022864
19,Lá Bèo sen dùng đắp ngoài thường để làm gì?,0.271878,1.0,0.357533,-0.285714,0.015695
30,Tên khoa học của cây Bầu là gì?,0.120090,0.0,-0.020016,0.500000,0.000378
20,Lá cây Đào tiên (trường sinh) thường dùng thế nào để chữa đau lưng?,0.103817,0.0,-0.001397,0.416667,0.000000



>>> TOP 5 GIẢM SÚT NHIỀU NHẤT (dựa trên trung bình các metrics):


,user_input,avg_diff,context_recall_diff,answer_correctness_diff,faithfulness_diff,answer_relevancy_diff
6,Côn bố còn được gọi bằng những tên nào?,-0.885820,-1.0,-0.575182,-1.000000,-0.968098
15,Hoa Cúc bách nhật còn dùng chữa bệnh vùng họng gì?,-0.812497,-1.0,-0.520094,-1.000000,-0.729894
16,Hoa Cúc bách nhật được dùng chữa bệnh hô hấp nào?,-0.808286,-1.0,-0.673915,-0.750000,-0.809228
22,Những người có thể trạng nào thường kỵ dùng Côn bố?,-0.662200,-1.0,-0.536955,-0.333333,-0.778511
11,Cụm hoa Actisô có những nhóm chất nào?,-0.542644,-1.0,0.010811,-1.000000,-0.181388


In [31]:
# ==== 6.2) Ví dụ chi tiết: Đối chiếu câu trả lời ====

if n_common > 0:
    # Chọn metrics quan trọng nhất để xem ví dụ
    target_metric = "answer_correctness"
    
    # Lấy 1 ví dụ cải thiện và 1 ví dụ giảm sút
    best_ex = diff_df.sort_values(f"{target_metric}_diff", ascending=False).iloc[0]
    worst_ex = diff_df.sort_values(f"{target_metric}_diff", ascending=True).iloc[0]
    
    def print_side_by_side(row, title):
        print(f"{"="*20} {title} {"="*20}")
        print(f"QUESTION: {row['user_input']}\n")
        print(f"[BASELINE RESPONSE]:\n{row['response_baseline']}\n")
        print(f"[PROPOSED RESPONSE]:\n{row['response_proposed']}\n")
        print(f"--- Metrics Diff ({title}) ---")
        for m in METRICS:
            diff = row[f"{m}_diff"]
            print(f"  {m:18}: {diff:+.4f} (Prop: {common.loc[row.name, f'{m}_proposed']:.4f} vs Base: {common.loc[row.name, f'{m}_baseline']:.4f})")
        print("\n" + "="*60 + "\n")

    print(f"--- VÍ DỤ CỤ THỂ DỰA TRÊN {target_metric} ---")
    print_side_by_side(best_ex, "CẢI THIỆN TỐT NHẤT")
    print_side_by_side(worst_ex, "GIẢM SÚT NHIỀU NHẤT")
else:
    print("Không có câu hỏi chung.")


--- VÍ DỤ CỤ THỂ DỰA TRÊN answer_correctness ---
==================== CẢI THIỆN TỐT NHẤT ====================
QUESTION: Theo y học cổ truyền, Hải tảo có tác dụng gì đối với đờm?

[BASELINE RESPONSE]:
Theo y học cổ truyền, Hải tảo có tác dụng nhuyễn kiên (làm mềm chất cứng), tiêu đàm (loại bỏ đờm), lợi thủy (thúc đẩy nước trong cơ thể ra ngoài), và trừ nhiệt (giảm nhiệt trong cơ thể). Đặc biệt, Hải tảo được sử dụng để điều trị các tình trạng liên quan đến đờm như ung thư tuyến giáp trạng, viêm phế quản mãn tính, bệnh mạch vành, và các tình trạng tích tụ đờm khác.

[PROPOSED RESPONSE]:
Theo y học cổ truyền, Hải tảo có tác dụng tiêu đàm. 

Nguồn: id=data\raw\cay-thuoc-vi-thuoc-phong-chua-benh-noi-tiet\cay-thuoc-vi-thuoc-phong-chua-benh-noi-tiet-p3.md:#4

--- Metrics Diff (CẢI THIỆN TỐT NHẤT) ---
  context_recall    : +0.0000 (Prop: 1.0000 vs Base: 1.0000)
  answer_correctness: +0.5852 (Prop: 0.8972 vs Base: 0.3120)
  faithfulness      : +0.5000 (Prop: 1.0000 vs Base: 0.5000)
  answer_rele

In [32]:
# ==== 6.3) Bảng tổng kết số lượng Cải thiện/Giảm sút ====
comparison_stats = []
for m in METRICS:
    better = (diff_df[f"{m}_diff"] > 0.001).sum()
    worse = (diff_df[f"{m}_diff"] < -0.001).sum()
    equal = n_common - better - worse
    comparison_stats.append({
        "Metric": m,
        "Better (Items)": better,
        "Worse (Items)": worse,
        "Equal (Items)": equal,
        "% Improved": round(better / n_common * 100, 2) if n_common > 0 else 0
    })

df_compare_stats = pd.DataFrame(comparison_stats).set_index("Metric")
display(df_compare_stats)


,Better (Items),Worse (Items),Equal (Items),% Improved
Metric,,,,
context_recall,2,10,19,6.45
answer_correctness,13,17,1,41.94
faithfulness,6,10,15,19.35
answer_relevancy,7,17,7,22.58
